In [1]:
import pandas as pd
import numpy as np


import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

import warnings

warnings.filterwarnings('ignore')

In [2]:
# leemos el csv con la data ya limpia y preprocesada
imdbesmod = pd.read_csv(r'../data/imdb_espaniol_mod.csv', sep=',')

In [3]:
# aislamos las columnas a utilizar y revisamos los tipos de datos
imdbesmodcopy = imdbesmod[['sentiemiento_token', 'review_es_mod']]
imdbesmodcopy['sentiemiento_token'] = imdbesmodcopy['sentiemiento_token'].astype('Int64')

In [4]:
# 'revolvemos' el dataframe y lo dividimos en test y train
imdbesmodcopy = imdbesmodcopy.sample(frac=1)
imdbesmodcopy_train = imdbesmodcopy.iloc[2500:]
imdbesmodcopy_test = imdbesmodcopy.iloc[:2500]

In [5]:
# creamos el tokenizador con 10000 palabras como maximo
max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(imdbesmodcopy_train['review_es_mod'])

In [6]:
# cambiando el texto a secuencias para su posterior 'embedding'
X_train = tokenizer.texts_to_sequences(imdbesmodcopy_train['review_es_mod'])
X_test = tokenizer.texts_to_sequences(imdbesmodcopy_test['review_es_mod'])

# pad sequences
maxlen = max([len(x) for x in X_train])

print('Max Length:', maxlen)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=maxlen, padding='post')

Max Length: 1870


In [7]:
# dividimos las etiquetas en train y test
y_train = imdbesmodcopy_train['sentiemiento_token'].values
y_test = imdbesmodcopy_test['sentiemiento_token'].values

In [8]:
# validacion y evaluacion
validation_split = 0.5
validation_size  =int(len(X_test) * validation_split)

X_val = X_test[:validation_size]
y_val = y_test[:validation_size]

X_eval = X_test[validation_size:]
y_eval = y_test[validation_size:]

In [9]:
# definimos la dimensionalidad del embedding
embed_size = 128

In [10]:
# Definimos el modelo y su arquitectura
model = Sequential([
    Embedding(max_features, embed_size),
    LSTM(60, return_sequences=True),
    GlobalMaxPool1D(),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(2, activation='sigmoid')
])

In [11]:
# lo compilamos
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# entrenamos el modelo
batch_size = 128
epochs = 30

early_stoppping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stoppping])

AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
Epoch 1/30
372/372 ━━━━━━━━━━━━━━━━━━━━ 2114s 6s/step - accuracy: 0.7404 - loss: 0.4864 - val_accuracy: 0.8936 - val_loss: 0.2629
Epoch 2/30
372/372 ━━━━━━━━━━━━━━━━━━━━ 2384s 6s/step - accuracy: 0.9144 - loss: 0.2192 - val_accuracy: 0.8824 - val_loss: 0.3042
Epoch 3/30
372/372 ━━━━━━━━━━━━━━━━━━━━ 2487s 7s/step - accuracy: 0.9401 - loss: 0.1641 - val_accuracy: 0.8880 - val_loss: 0.2970
Epoch 4/30
372/372 ━━━━━━━━━━━━━━━━━━━━ 2489s 7s/step - accuracy: 0.9678 - loss: 0.1008 - val_accuracy: 0.8808 - val_loss: 0.3390


In [13]:
# vemos el score del modelo
score = model.evaluate(X_eval, y_eval, batch_size=batch_size)

print(f'Test loss: {score[0]} - Test accuracy: {score[1]}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 831ms/step - accuracy: 0.8833 - loss: 0.2904
Test loss: 0.30798858404159546 - Test accuracy: 0.8751999735832214


In [16]:
# guardamos el modelo en un archivo .h5 asi como sus pesos y el tokenizer
import json
import keras

## Modelo
#model.save(r'../model/csModel.h5')
keras.saving.save_model(model, 'my_model.keras')

## guardamos los pesos (weights)
model.save_weights(r'../model/csModel.weights.h5')

## Tokenizador
with open(r"../model/cs_tokenizer.json", "w") as j:
    json.dump(tokenizer.to_json(), j)


In [39]:
model.save('my_model.h5')

In [ ]:
'''
# utilizando earlystopping
import time
import matplotlib.pyplot as plt

# Definir el modelo con EarlyStopping
def build_model():
    model = Sequential([
        Embedding(max_features, embed_size),
        LSTM(60, return_sequences=True),
        GlobalMaxPool1D(),
        Dense(50, activation='relu'),
        Dropout(0.1),
        Dense(2, activation='sigmoid')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Función para probar diferentes combinaciones de hyperparámetros
def evaluate_model(X_train, y_train, X_val, y_val, batch_sizes, epochs_list):
    results = []
    
    for batch_size in batch_sizes:
        for epochs in epochs_list:
            # EarlyStopping: Detener entrenamiento si no hay mejoras en 3 épocas
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            
            # Construir y entrenar el modelo
            model = build_model()
            
            print(f"Entrenando modelo con batch_size={batch_size} y epochs={epochs}")
            
            # Medir el tiempo de entrenamiento
            start_time = time.time()
            
            history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                                validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)
            
            end_time = time.time()
            training_time = end_time - start_time
            
            # Guardar resultados
            val_accuracy = history.history['val_accuracy'][-1]
            results.append({
                'batch_size': batch_size,
                'epochs': epochs,
                'val_accuracy': val_accuracy,
                'training_time': training_time
            })
            
            print(f"Completado: val_accuracy={val_accuracy:.4f}, tiempo={training_time:.2f} segundos")
    
    return results
'''

In [ ]:
# Definir los tamaños de lote y el número de épocas que quieres probar
#batch_sizes = [32, 64, 128]
#epochs_list = [10, 30, 50]

In [ ]:
# Evaluar el modelo
#results = evaluate_model(X_train, y_train, X_val, y_val, batch_sizes, epochs_list)

#Entrenando modelo con batch_size=32 y epochs=10
#AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
#Completado: val_accuracy=0.8800, tiempo=2996.96 segundos
#Entrenando modelo con batch_size=32 y epochs=30
#Completado: val_accuracy=0.8848, tiempo=3750.51 segundos
#Entrenando modelo con batch_size=32 y epochs=50
#Completado: val_accuracy=0.8832, tiempo=3706.10 segundos
#Entrenando modelo con batch_size=64 y epochs=10
#Completado: val_accuracy=0.8808, tiempo=4486.20 segundos
#Entrenando modelo con batch_size=64 y epochs=30

In [ ]:
results = [
    {
        'batch_size': 32,
        'epochs': 10,
        'val_accuracy': 0.8800,
        'training_time': 2996.96
    },
    {
        'batch_size': 32,
        'epochs': 30,
        'val_accuracy': 0.8848,
        'training_time': 3750.51
    },
    {
        'batch_size': 32,
        'epochs': 50,
        'val_accuracy': 0.8832,
        'training_time': 3706.10
    },
    {
        'batch_size': 64,
        'epochs': 10,
        'val_accuracy': 0.8808,
        'training_time': 4486.20
    },
]
# Mostrar los resultados
for result in results:
    print(result)

In [ ]:
import matplotlib.pyplot as plt
# Opcional: Graficar la relación entre batch_size, epochs y val_accuracy
batch_sizes_vals = [result['batch_size'] for result in results]
epochs_vals = [result['epochs'] for result in results]
val_accuracies = [result['val_accuracy'] for result in results]

plt.figure(figsize=(10, 6))
plt.scatter(epochs_vals, batch_sizes_vals, c=val_accuracies, cmap='viridis', s=100)
plt.colorbar(label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Batch Size')
plt.title('Evaluación de Hyperparámetros (Batch Size vs Epochs vs Val Accuracy)')
plt.show()

## probando el modelo 

In [17]:
prueba = [
    "El café en este lugar es increíblemente aromático y el ambiente es perfecto para relajarse. ¡Muy recomendado!",
    "El servicio fue lento y las porciones pequeñas para el precio. No lo recomendaría.",
    "El libro tiene una trama cautivadora y personajes memorables. Me mantuvo enganchado de principio a fin.",
    "La película fue predecible y los efectos especiales parecían anticuados. Me decepcionó.",
    "Este champú dejó mi cabello suave y brillante, y el aroma es maravilloso. Definitivamente lo volveré a comprar.",
    "La aplicación se traba constantemente y consume mucha batería. No vale la pena descargarla.",
    "La chaqueta es súper cómoda y tiene un diseño moderno. Perfecta para el invierno.",
    "La comida llegó fría y con un sabor mediocre. No volveré a pedir de aquí.",
    "El curso es muy claro y los ejemplos son prácticos. Aprendí muchísimo en poco tiempo.",
    "Las instrucciones no eran claras y el producto vino defectuoso. Un desperdicio de dinero."
]
sentimiento = [
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo'
]
resenias = pd.DataFrame({'review_es': prueba, 'sentimiento': sentimiento})
resenias = resenias.sample(frac=1)

In [19]:
import re
from nltk.corpus import stopwords
import spacy
# APLICADO A COLUMNAS

# Función para eliminar etiquetas HTML de una columna
def eliminar_etiquetas_html(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    df[col_out] = df[col_in].apply(lambda x: re.sub(r'<.*?>', '', x) if isinstance(x, str) else x)
    return df

# Función para limpiar caracteres especiales de una columna
def limpiar_texto(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    df[col_out] = df[col_in].apply(lambda x: re.sub(r'[^\w\s.,]', '', x) if isinstance(x, str) else x)
    return df

# ------------->
# correción de funcion para limpiar caracteres especiales
def caracteres_especiales(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    df[col_out] = df[col_in].apply(lambda x: re.sub(r'[^\w\s]', ' ', x) if isinstance(x, str) else x)
    return df

def espacios_extra(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    df[col_out] = df[col_in].apply(lambda x: re.sub(r'\s+', ' ', x).strip() if isinstance(x, str) else x)
    return df
# ------------->

# Función para eliminar palabras vacías de una columna
def remove_stop_words(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    stop_words = set(stopwords.words('spanish'))
    df[col_out] = df[col_in].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]) if isinstance(x, str) else x)
    return df

def lemmatizador(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    # cargamos el modelo en espaniol
    nlp = spacy.load('es_core_news_sm')

    # aplicamos a cada fila de la columna entrada
    df[col_out] = df[col_in].apply(
        lambda x: ' '.join([token.lemma_ for token in nlp(x)]) if isinstance(x, str) else x
    )
    return df

In [20]:
def preprocessing_pipe(df : pd.DataFrame, col_in : str, col_out : str) -> pd.DataFrame:
    result = (df
              .pipe(eliminar_etiquetas_html, col_in, col_out)
              .pipe(caracteres_especiales, col_out, col_out)
              .pipe(espacios_extra, col_out, col_out)
              .pipe(remove_stop_words, col_out, col_out)
              .pipe(lemmatizador, col_out, col_out))
    return result

In [22]:
resenias_treated = preprocessing_pipe(resenias, 'review_es', 'review_es_mod')

In [23]:
maxlen

1870

In [24]:
# cambiando el texto a secuencias para su posterior 'embedding'
x_review_tests = tokenizer.texts_to_sequences(resenias_treated['review_es_mod'])
x_review_tests = sequence.pad_sequences(x_review_tests, maxlen=maxlen, padding='post')

In [26]:
prediccion = model.predict(x_review_tests)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step


In [28]:
prediccion_clase = prediccion.argmax(axis=-1)

In [29]:
prediccion_clase

array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0], dtype=int64)

In [31]:
resenias_treated['prediccion'] = prediccion_clase

In [34]:
resenias_treated['prediccion'] = resenias_treated['prediccion'].map({1 : 'positivo', 0 : 'negativo'})

In [38]:
resenias_treated

,review_es,sentimiento,review_es_mod,prediccion
0,El café en este lugar es increíblemente aromát...,positivo,café lugar increíblemente aromático ambiente p...,positivo
5,La aplicación se traba constantemente y consum...,negativo,aplicación trar constantemente consumir mucho ...,positivo
6,La chaqueta es súper cómoda y tiene un diseño ...,positivo,chaqueta súper cómodo diseño moderno Perfecta ...,positivo
1,El servicio fue lento y las porciones pequeñas...,negativo,servicio lento porción pequeño precio recomendar,negativo
4,"Este champú dejó mi cabello suave y brillante,...",positivo,champú dejar cabello suave brillante aroma mar...,positivo
7,La comida llegó fría y con un sabor mediocre. ...,negativo,comida llegar fría sabor mediocre volver pedir...,negativo
3,La película fue predecible y los efectos espec...,negativo,película predecible efecto especial parecer an...,negativo
8,El curso es muy claro y los ejemplos son práct...,positivo,curso claro ejemplo práctico Aprendí muchísimo...,negativo
2,El libro tiene una trama cautivadora y persona...,positivo,libro trama cautivadora personaje memorable ma...,positivo
9,Las instrucciones no eran claras y el producto...,negativo,instrucción claro producto venir defectuoso de...,negativo


### Prueba con un dataset mayor
Dataset: ./model/netflix/film_reviews_result.csv

In [ ]:
# importamos los datos
netflix = pd.read_csv(r'./model/netflix/film_reviews_result.csv', sep='|')